# `neo4j`

## graph databases

in the previous database lectures we covered relational databases (specifically, the `aws` `rds` service and `postgres` as a `rdbm`), and a particular type of `nosql` database (`aws`'s proprietary cloud-based key-value / document store `dynamodb`). I also mentioned that there were several other types of `nosql` databases. among those other types, there is one that is gaining a lot of prominence:  **graph** databases.

to satisfy my own curiosity: by show of hands, who here has previous experience with:

1. graphs (as a general concept, *e.g.* in MATH 442)
2. graph calculations (*e.g.* betweenness, page rank)
2. graph databases (`neo4j`, `titan`, `cosmos db`)

graph databases are having a bit of a moment. it's a hype moment, in part, but a moment nevertheless.

as a data scientist, I think this is *awesome*. graph databases have immediate implications in a handful of interesting data science problems. before we get into that, though...

### what is the problem?

why did anyone ever go about creating graph databases in the first place?

I have previously been framing the difference between traditional `rdbm`s and more modern `sql` as being "relational vs. non-relational". it's important to remember here that 

<div align="center">*relational `!=` relationship*</div>

remember: "relational" effectively means normalized tables

as it so happens, both relational databases and most `nosql` (*e.g.*, document stores, key-value stores, columnar) databases are fundamentally ill suited for analyzing the "interconnected-ness" of data -- a concept that is often the primary driver in questions data scientists are interested in asking (*e.g.* fraud rings in insurance claims)

#### a motivating example: relational databases

let's think about the way we would choose to model the following data:

+ movies
+ actors, writers, producers, and directors
+ reviewers

suppose the information is basically already collected for you. how might you organize these concepts in a *relational database*?

In [ ]:
import pandas as pd
    
base = 'https://s3.amazonaws.com/shared.rzl.gu511.com/neo4j'                                                                                 
                                                                                                                                            
actedin = pd.read_csv('{}/{}'.format(base, 'acted_in.csv'))                                                                                
directed = pd.read_csv('{}/{}'.format(base, 'directed.csv'))                                                                                
movie = pd.read_csv('{}/{}'.format(base, 'movie.csv'))                                                                                      
person = pd.read_csv('{}/{}'.format(base, 'person.csv'))                                                                                    
produced = pd.read_csv('{}/{}'.format(base, 'produced.csv'))                                                                                
reviewed = pd.read_csv('{}/{}'.format(base, 'reviewed.csv'))                                                                                
wrote = pd.read_csv('{}/{}'.format(base, 'wrote.csv'))

In [ ]:
person.head()

In [ ]:
movie.head()

In [ ]:
actedin.head()                                                                                                                              

In [ ]:
directed.head()                                                                                                                             

In [ ]:
produced.head()                                                                                                                             

In [ ]:
reviewed.head()                                                                                                                             

In [ ]:
wrote.head()

as it just so happens, I already put those very dataframes into tables with the same names on our shared `rds postgres` service -- this should be ***easy***

so now suppose our goal is to play the [six degrees of Kevin Bacon](https://en.wikipedia.org/wiki/Six_Degrees_of_Kevin_Bacon) game.

let's start with something simple: just find every pair of people who have acted together in the same movie and the name of that movie

no problem in `sql`, right?

In [ ]:
qry = """
SELECT
    p1.name AS p1_name
    , m.title as m_title
    , p2.name AS p2_name
FROM
    person AS p1
    JOIN acted_in AS a1
    ON
        p1.id = a1.person
    JOIN movie AS m
    ON
        a1.movie = m.id
    JOIN acted_in AS a2
    ON
        m.id = a2.movie
        AND a1.person != a2.person
    JOIN person AS p2
    ON 
        a2.person = p2.id
"""

In [ ]:
import psycopg2
import getpass

host, port, user, dbname = ('rzl-gu511-shared.cdmknaubrmaw.us-east-1.rds.amazonaws.com', 5432, 'gu511', 'gu511')
password = getpass.getpass('password: ')

with psycopg2.connect(host=host, port=port, user=user, dbname=dbname, password=password) as conn:
    partners = pd.read_sql(qry, conn)
partners.tail(15)

simple...

now how about people who have worked in a movie with someone who worked in a movie with someone else?

In [ ]:
qry = """
SELECT
    p1.name AS p1_name
    , m1.title AS m1_title
    , p2.name AS p2_name
    , m2.title AS m2_title
    , p3.name AS p3_name
FROM
    person AS p1
    JOIN acted_in AS a1
    ON
        p1.id = a1.person
    JOIN movie AS m1
    ON
        a1.movie = m1.id
    JOIN acted_in AS a2
    ON
        m1.id = a2.movie
        AND a1.person != a2.person
    JOIN person AS p2
    ON 
        a2.person = p2.id
    JOIN acted_in AS a3
    ON
        p2.id = a3.person
    JOIN movie AS m2
    ON
        a3.movie = m2.id
    JOIN acted_in AS a4
    ON
        m2.id = a4.movie
        AND a3.person != a4.person
        AND a1.person != a4.person
    JOIN person AS p3
    ON 
        a4.person = p3.id
"""

In [ ]:
with psycopg2.connect(host=host, port=port, user=user, dbname=dbname, password=password) as conn:
    partners = pd.read_sql(qry, conn)
partners.head(10)

wanna keep going?

how about we add in non-actors (i.e. join on `acted_in` and `directed` and `writed` and `produced`)?

it may seem bad, but in reality, it's actually worse than it seems.

those self joins are often *exponentially* expensive (think about friends of friends of friends on facebook). every extra node makes our query more difficult to process

also, as confusing as constructing these queries was (and we could have changed our database designe to facilitate them better, for sure), we still end up performing another self join every time we want to hop.

`sql` is great for one-to-one or one-to-many sorts of relationships, but less so for one-to-one-to-many-to-one-to-many-to-many-to-one-to...

in summary: the problem here isn't so much the relational database as it is the problem we are trying to solve with the relational database.

#### a motivating example: `nosql` databases

the situation isn't much better in `dynamodb`-world. remember, the price of distributed servers and super-fast lookups in our key-value / document stores was that we could basically *only* perform super-fast lookups on primary keys.

suppose that in the above movie example we had encoded movies as:

In [ ]:
movie = {
    'released': 1999,       # partition / hash key
    'title': 'The Matrix',  # sort key
    'tagline': 'Welcome to the Real World',
    'actors': [
        {'roles': 'Neo', 'name': 'Keanu Reeves'},
        {'roles': 'Trinity', 'name': 'Carrie-Ann Moss'},
        {'roles': 'Morpheus', 'name': 'Laurence Fishburne'},
        {'roles': 'Agent Smith', 'name': 'Hugo Weaving'},
        {'roles': 'Emil', 'name': 'Emi Eifrem'},
    ],
    'producers': ['Joel Silver'],
    # no writers attribute for this node
    'directors': ['Lilly Wachowski', 'Lana Wachowski']
}

if we encoded individual movies in this way, we could quickly look up all movies released in a given year, or a specific movie. but if we wanted to do anything with the actors, we are already out of luck: we can't index on the compound attributes, but even if we could, we're still stuck performing "a lookup, then a scan, then a lookup, then a scan, etc." for all of the hops in our query

#### what to do?

these scenarios may feel contrived, but they are actually an incredibly common scenario to care about and want to model:

+ what is the shortest path between two people in a social network? 
    + can we determine how they met? or who connected them?
+ are there secret, back-channel conversation pathways or financial flows?
+ are certain actors in a network of people exerting more influence, or suddently exerting influence where they weren't before?
+ are individual group leaders critical to the communication chain and survival of a hostile movement?
+ do certain words always appear within some number of eachother, or in certain orders, in texts indicating securities fraud?

relational databases aren't good at handling complicated relationships and networks, and our other `nosql` options often struggle to handle relationships between records at all (by design).

because of the ubiquity of these enherently graph-like problems, a special class of databases was created to specifically model graphs

### what is a graph database?

a graph database is a `nosql` ("non-relational", "no tables") database which approaches all data storage as being of basically two types: *nodes* and *relationships*

#### nodes

a node is, extremely generically speaking, "a thing." nodes usually represent the high-level concepts you first think about when you're talking about a problem domain. they are concepts which are connected in some yet-defined way to otehr concepts.

they are usually the things which we describe in our language with nouns, but they don't have to be.

a graph model distinguishes between nodes in two basic ways: *labels* and *properties*.

first, a single graph can have nodes of many different "types" of things, in much the same way our understanding of a problem involves multiple concepts.

the technical term we will use for the "type" of node is the node's *label*. a single node can represent zero, one, or more "types" of things, and can have zero, one, or more *labels*.

for example, in the model above we could say that we have `Movie` and `Person` as two types of entites, and we could have nodes labelled `Movie` (a thing that is a movie) and nodes labelled `Person` (a thing that is a person).

for example, we could have another concept, such as [SAG](https://en.wikipedia.org/wiki/Screen_Actors_Guild) member, and `SAG` could be another label on a node with a `Person` label.

what we said above applies to a single, generic node. a graph is composed of possibly many nodes, and possibly many with the same *label*. 

individual nodes have *properties*: properties that describe them in greater detail. as was the case with `dynamodb`, there is no schema to the *properties* of a node -- anything goes. there is, generically, a demand that there be one unique identifier, but that is usually handled by the database itself.

one node *labelled* `Person` may have the properties

```json
{
    'name': 'Keanu Reeves',
    'born': 1964
}
```

in summary,

1. nodes are categorized into broad groups by their *labels*
2. individual nodes are describe by their *properties*

schematically, nodes are often written in a sort of pictoral way (more on this later):

```cypher
(p:Person {name: 'Keanu Reeves', born: 1954})
```

#### edges 

relationships between nodes in our graph are called *edges*.

because ndoes are representing relationships and connections between nodes (generally nouns), it is common for them to be representing actions (generally verbs). those verbs themselves are active or passive, which results in *direction* of the edges -- all edges in these graphs are *directed*

take our movie example. Keanu Reeves was an actor in The Matrix, playing a character named Neo. we could create an edge between the `Person` node representing Keanu Reeves and the `Movie` node representing "The Matrix".

as with nodes, edges have two sorts of distinctions: the *type* and *properties*

these two distinctions are analogous to the distinction types for nodes:

1. edges are categorized into broad groups by their *types*
    + while nodes can have many labels, edges can have **one and only one** *type*
2. individual edges are describe by their *properties*
    + usually quantitative things (weights, time lengths)

there is a similarly pictoral `ascii` reprentatino of edges:

```cypher
(p:Person)-[a:ACTED_IN {weight: 2.5}]->(m:Movie)
```

note, in particular, that *edge types* are all-caps, and the arrows are written as directed (as if you were reading the relationship out loud)

##### to label, relate, or "propertize"...

up above, we discussed another concept which might seem like a good candidate for a *node label*: the role a person had in a film (actor, director, producer, writer).

for example, Clint Eastwood has been both an actor and a director. we could have created *node labels* "Actor" and "Director" and treated the Clint Eastwood node as having *labels* `Person, Actor, Director`. that would be a totally valid graph model.

```cypher
(p:Person:Actor:Director {name: 'Clint Eastwood'})
```

however, consider: Clint Eastwood is *always* a person, but he is not *always* an Actor, nor is he *always* a Director: he takes on different combinations of roles for different movies.

in this sense, it could be that the role is something we treat as a relationship between Clint Eastwood the Person node and the Movie nodes of which he has been a part (e.g "Unforgiven").

```cypher
(p:Person {name: 'Clint Eastwood'})-[:ACTED_IN {role: 'William Munny'}]->(m:Movie {name: 'Unforgiven'})

(p:Person {name: 'Clint Eastwood'})-[:DIRECTED]->(m:Movie {name: 'Unforgiven'})
```

#### the cost of putting your relationships first

graph databases elevate the relationships -- basically, those `JOIN` conditions in our complicated `sql` statement above -- into first-class properties of every node (record). this means that accessing those related neighbor nodes and traversing those relationships is *very* fast, and easy to visualize / write.

that being said, all of this is predicated on looking at relationships *from some starting point*. generally speaking, graph database calculations are very fast when *anchored* on a starting node, and if you can formulate your problem such that it is anchored this way, that will be better.

there are *plenty* of network calculations that can't be isolated to single nodes, and require the entire graph (e.g. graph diameter, page rank).

### what graph database options are out there?

#### semantics

it's common for people to discuss different concepts in the graph database world with the same language, and to often be a little careless when doing so.

there are three different types of "technologies" commonly discussed in the context of graph databases:

1. **graph databases**: the software that store, model, and represent data as nodes and edges in a graph
2. **query languages and graph apis**: domain-specific languages (both theoretical and implemented) that can be used to interact with graph database software
3. **graph and network calculation libraries**: code implementations of algorithms and metrics which leverage the above

some of these semantic distinctions exist in the relational database world as well: `postgres` is a database, `sql` is a query language. the calculation libraries are specific to the data type, so the analogy is not as well defined.

##### databases

there are [a ton of options](https://en.wikipedia.org/wiki/Graph_database#List_of_graph_databases) out there, and the number will likely grow with the hype.

among them, `neo4j` is [the clear leader](https://db-engines.com/en/ranking), with Microsoft's Azure Cosmos DB in a rising second. we will focus on `neo4j`.

for perspective, it's also useful to [compare `neo4j` to relational and other `nosql` databases](https://db-engines.com/en/ranking/) (24th, between `sap hana` and `dynamodb`)

##### query languages and `api`s

`sql` is an abstract language designed for working with relations (tables), and it gets implemented in individual `rdbm`s. 

in a similar way, people are actively developing the `sql` of the graph database world. no one language has been selected, but [`cypher`](https://neo4j.com/developer/cypher-query-language/) (the query language of `neo4j`) is an early candidate.

[other alternatives exist](https://en.wikipedia.org/wiki/Graph_database#APIs_and_graph_query-programming_languages), and of them `graphql` (facebook) and `gremlin` are the most common

##### graph calculation and visualization libraries

finally, calculation libraries have cropped up as graph and network analysis has become more important. some require loading an entire graph into memory where all calculations are done, and some are written utilizing the above items to perform *ad hoc* queries and limit memory constraints.

a large segment of this space focuses primarily on *visualization* of large graphs

popular libraries include:

+ [`networkx`](https://networkx.github.io/): for calculation and limited vis
+ [`igraph`](http://igraph.org/redirect.html): same
+ [`stinger`](http://www.stingergraph.com/): an impressively fast graph algorithm library (implements its own properietary graph storage structure)
+ [`gephi`](https://gephi.org/): excellent visualization studio with some surprising calculation abilities
+ [`SNAP` (stanford network analysis platform)](https://snap.stanford.edu/snap/): bills itself as being the best network analytics package for very large (millions of nodes, billions of edges, which *is* quite large) networks. I haven't tried it
+ [`sigma.js`](http://sigmajs.org/): a very prominent `javascript` graph rendering and visualization library (no computation to speak of)
+ [`cytoscape`](http://www.cytoscape.org/) and `cytoscape.js`: *the* network library for the molecular biology community, mostly vis

## modelling

the concept of a graph is so generalizable that just about any data *could* be cast as a graph, so go nuts.

that being said, there are some scenarios that immediately lend themselves to graph modelling

first, basically any time you have people interactings with eachother (and you care more about the interactions than the people). this includes

+ [social networks](https://neo4j.com/use-cases/social-network/)
+ [recommender systems](https://neo4j.com/use-cases/real-time-recommendation-engine/)

another huge area of interest has been financial 

+ fraud ([in banks](https://neo4j.com/graphgist/9d627127-003b-411a-b3ce-f8d3970c2afa?ref=solutions), [in insurance](https://neo4j.com/blog/insurance-fraud-detection-graph-database/))
+ financial networks and money laundering
    + [the panama papers](https://panamapapers.icij.org/) (available as a `neo4j` sandbox)
    + [the paradise papers](https://www.icij.org/investigations/paradise-papers/explore-politicians-paradise-papers/)

they are also gaining a lot of popularity in areas which focus on resource management and connectedness, such as

+ [identity and access management systems](https://neo4j.com/use-cases/identity-and-access-management/)
+ large-organization data management ([knowledge graphs](https://neo4j.com/use-cases/knowledge-graph/), [data provenance systems](https://neo4j.com/use-cases/master-data-management/))
+ [IT architecture and critical asset identification](https://neo4j.com/use-cases/network-and-it-operations/)

finally, they can be useful for sequential data where items are reused or nested:

+ [text analytics](https://graphaware.com/neo4j/2016/07/07/mining-and-searching-text-with-graph-databases.html)
+ [time series analysis](https://github.com/SocioPatterns/neo4j-dynagraph/wiki/Representing-time-dependent-graphs-in-Neo4j)

### example model

<div align="center">**walkthrough: an example model in the `neo4j` sandbox**</div>

let's look at how that movie data model we discussed above can be represented in a graph. start by heading to [the `neo4j` sandbox site](https://neo4j.com/sandbox-v2/)

the walkthrough goes through the following steps:

1. create a `neo4j` sandbox
    1. open [the `neo4j` sandbox site](https://neo4j.com/sandbox-v2/)
    2. log in
    3. under the "Launch a New Sandbox" header, find the "Neo4j 3.3" (*i.e.* "Blank Sandbox") option and click "Launch Sandbox"
    4. that will take some time to launch, so look at the other options as example models
    5. click on the link to the `neo4j` browser
2. run the "Movies" gist
    0. you could just type `:play movie-graph` and run it, **or**
    1. click on the Star icon in the left menu bar
    2. click on the "Example Graphs" menu item
    3. click on the "Movie Graph" element
    4. run (play button) the graph line that was brought up
    5. click right and run the first major code block
3. check out the movie graph relationship schema
    0. you could just type `CALL db.schema()` and run it, **or**
    1. click on the Star icon in the left menu bar
    2. click on the "What is related, and how" option

## cypher

in the previous walkthrough, we were able to use the `neo4j` web browser to auto-generate some query strings like the following:

```cypher
MATCH (tom:Person {name:"Tom Hanks"})-[:ACTED_IN]->(m)<-[:ACTED_IN]-(coActors),
      (coActors)-[:ACTED_IN]->(m2)<-[:ACTED_IN]-(cruise:Person {name:"Tom Cruise"})
RETURN tom, m, coActors, m2, cruise
```

this declarative query language was created by `neo4j` and is called `cypher`. it is one of several domain-specific query languages out there (discussed above), but the only one we will focus on at this time.

it has a lot of similarities to `sql`, as well as some obvious differences. let's discuss the basics. refer to [the documentation](https://neo4j.com/docs/developer-manual/current/cypher/) for total coverage of the query language

### patterns

`cypher` allows you to traverse the graph by matching text patterns against graph structures: if you describe a graph structure using `cypher` shorthand *patterns*, `neo4j` will look for all segments of the graph which match that *pattern*.

#### `MATCH`ing patterns
matching patterns is so important in `cypher` that is how almost every query starts. think of `MATCH` as the `SELECT` of `cypher`.

the most basic query syntax is

```cypher
MATCH ...  // ** some pattern here **
RETURN ... // ** some properties, or calculations on the elements matched by that pattern **
```

#### pattern pieces: nodes, edges, and paths

to start with, the main objects in the graph database (nodes and edges) have plain-text symbolic representations (like we discussed above):

```cypher
// nodes
(nodeavariable:NodeLabel {nodeProperty: 'nodePropertyValue'})

// edges
()-[edgevariables:EdgeLabel {edgeProperty: 'edgePropertyValue'}]->()
```

the first word inside these symbolic representations is a node or edge *variable*. this is analogous to aliasing in `sql`:

```sql
SELECT
    t1.firstname AS name
...
```

you can use variables in the parts of the query that follow (*e.g.* the `RETURN` statement) to reference the items matching that symbolic pattern of that node or that edge

*paths* -- collections of connected nodes and the edges which connect them -- can be expressed by chaining the above elements together, as was done in the Tom Hanks meets Tom Cruise query:

```cypher
MATCH (hanks:Person {name:"Tom Hanks"})-[:ACTED_IN]->(m)<-[:ACTED_IN]-(coActors),
      (coActors)-[:ACTED_IN]->(m2)<-[:ACTED_IN]-(cruise:Person {name:"Tom Cruise"})
RETURN tom, m, coActors, m2, cruise
```

```cypher
MATCH (hanks:Person {name:"Tom Hanks"})-[:ACTED_IN]->(m)<-[:ACTED_IN]-(coActors),
      (coActors)-[:ACTED_IN]->(m2)<-[:ACTED_IN]-(cruise:Person {name:"Tom Cruise"})
RETURN tom, m, coActors, m2, cruise
```

this path can be described by simply reading it:

+ a person named "Tom Hanks"... acted in... a movie
+ that movie was acted in by another person
+ that person acted in a movie
+ that movie was acted in by a person named "Tom Cruise"

generally speaking, you can leave out as much of the pattern as you want.

the only *truly* required things are the `()` and (if more than one node is involved), the `--` elements (I have intentionally left out the direction on that edge as well as the label; neither are *necessary* for a path represenation, though either are often good to include).

the simplest example is the pattern which matches any node:

```cypher
MATCH (n)
```

an only slightly more complicated example: the following would find all pairs of nodes of any type that are 4 edges away from eachother (a harder thing than the one we didn't even try in `sql` earlier), and assign them to variables `n0` and `n1`:

```cypher
MATCH (n0)--()--()--()--(n1)
```

this could (will) re-use any one path any number of times.

writing all of those edges can be tedious, so `cypher` has a shorthand for representing paths with several consecutive edges of the same type (requires you don't care about the nodes between):

```cypher
// any length
MATCH (n0)-[*]-(n1)
          
// exactly two edges
MATCH (n0)-[*2]-(n1)
```

```cypher
// 1, 2, or 3 edges
MATCH (n0)-[*1..3]-(n1)
              
// up to 5 edges
MATCH (n0)-[*..5]-(n1)

// 5 or more edges
MATCH (n0)-[*5..]-(n1)          
```

#### `RETURN`ing results

once you have matched a pattern and created variables from portions of it, you can `RETURN` those variables.

under the hood, this is (yet again) an `http` request. the response body is a `json` object describing the query result (check out the "`</>` Code" button in your query result box).

if what was returned is limited to node and edge objects, `neo4j`'s web browser will display them in the graphs we are familiar with. otherwise, they will be displayed as a table

for example, it's easy to get a visual graph of a fixed number (`LIMIT`) of nodes:

```cypher
MATCH (n)
RETURN n
LIMIT 10
```

be sure to check out the other tabs in your browser window

it's also easy to pack pattern objects into paths and return the *paths*:

```cypher
MATCH p=(n0)--(n1)
RETURN p
LIMIT 10
```

again, check out the other tabs in the browser window

as I mentioned above, you don't *have* to return variables (`n`, `p`) -- you can return properties as well. for example:

```cypher
MATCH (p:Person)-[a:ACTED_IN]->(m:Movie)
RETURN p.name, a.roles, m.title
```

*note*: the browser no longer renders these as graph objects *because they aren't objects*, they're *properties*

### operators

just like `sql`, `cypher` has several built-in [operators](http://neo4j.com/docs/developer-manual/current/cypher/syntax/operators/) -- reserved keywords that accomplish certain basic calculations, logical comparison, or control flow.

[this table](http://neo4j.com/docs/developer-manual/current/cypher/syntax/operators/#query-operators-summary) covers the list, and if you are familiar with `sql` you should be familiar with what all of these are doing.

### clauses

both `MATCH` and `RETURN` are important `cypher` keywords that designate that the words following them (in this case: patterns) should be interepreted in a certain way and `neo4j` should take a certain action. they are part of a broader class of special action or control flow keywords called *clauses*

allow me divide clauses up into two types:

1. those used to build queries and read data
2. those used to create and update data

#### query operations (read)

##### `MATCH`

the `MATCH` statement indicates that what follows should be treated as a pattern and `neo4j` should identify graph pieces that match that pattern.

##### `RETURN`

the `RETURN` statement instructs the query engine to take whatever variables or values are defined at that point in the query and limit them down to a certain structured subset

there are several other important *clauses* which we will cover at a high level -- read about the rest [here](http://neo4j.com/docs/developer-manual/current/cypher/clauses/).

##### `WHERE`

the `WHERE` clause only applies following a `MATCH` clause, and it does what you expect: it takes the list of sub-graphs matching a given pattern and filters them to those which satisfy some requirement

```cypher
MATCH (n:Person)-[a:ACTED_IN]->(m:Movie)
WHERE size(a.roles) > 1
RETURN n, a, m
```

##### `ORDER BY`, `SKIP`, and `LIMIT`

these three clauses all affect how the results of a pattern match query are returned, either by sorting them, skipping some number of them, or only returning the first number of them

##### `WITH`

this clause follows a `MATCH` query and uses the results of that query to "set the stage" for the next. basically, it allows you to chain several queries together. for example:

```cypher
MATCH (actor:Person)-[:ACTED_IN]->(m1:Movie)
WITH actor
MATCH (actor)-[:DIRECTED]->(m2:Movie)
RETURN actor, m2
```

##### `UNWIND`

this will take a list of items and *unwind* them into a sequence of rows. this is used to explode collections of things into separate records. For example:

```cypher
UNWIND [1, 2, 3, 4, 5, 6, 7] AS x
RETURN x
```

##### `UNION` and `UNION ALL`

just as in `sql`, this takes the results of multiple queries and combines them. the former removes duplicates and the latter keeps them

#### create and update clauses (write)

the following clauses are used to create new nodes and edges or to update existing ones

##### `CREATE`

create a node or relationship:

```cypher
// one node at a time
CREATE (n:TestLabel {x: 1})

// many elements in a path
CREATE p = (m1:MyLabel {msg: 'hello world'})-[e:MY_TYPE {x: 1}]->(m2:MyLabel {msg: 'goodbye world'})
RETURN p
```

##### `DELETE`, `DETACH DELETE`

you can `DELETE` and isolated node or any edge, or you can `DETACH DELETE` a connected node

```cypher
// isolated node
MATCH (n:TestLabel {x: 1})
DELETE n
                      
// will fail because node isn't detached
MATCH (m1:MyLabel {msg: 'hello world'})
DELETE m1
                     
// will detach node then delete it
MATCH (m1:MyLabel {msg: 'hello world'})
DETACH DELETE m1
```

##### `SET` and `REMOVE`

the `SET` clause allows you to set labels on nodes or properties on nodes and edges, and `REMOVE` lets you undo that operation

```cypher
MATCH (e:Person {name: "Emil Eifrem"})
SET e.l33t = true, e:NeoFounder, e:BigHollywoodActor
RETURN e
```

```cypher
MATCH (e:Person {name: "Emil Eifrem"})
REMOVE e:BigHollywoodActor
RETURN e
```

##### `MERGE` [`ON CREATE`, `ON MATCH`]

if you are familiar with the `sql` concept of an `UPSERT` (an update or an insert), this is the `cypher` equivalent. basically, this statement looks to match a given pattern, and if it doesn't find that pattern to create some nodes. `MERGE` on its own will do only this.

```cypher
MERGE (z:Teacher {name: "Zach Lamberty"})
RETURN z
```

there are times, however, where you want to *do* something when you create a node, and then you want to *do something different* when you are matching / updating. this is facilitated through the trailing clauses `ON CREATE` and `ON MATCH`:

```cypher
MERGE (c:TA {name: "Carlos Blancarte"})
ON CREATE SET c.created = timestamp()
ON MATCH SET c.lastMatched = timestamp()
RETURN c
```

##### `LOAD CSV`

no database would be worth a single durn thing if it couldn't perform bulk inserts of `csv`s, so let's cover that too.

every invocation of `LOAD CSV` needs to provide the file path or url of the `csv` file. `neo4j` then loads every record in that `csv` into:

1. a list (items accessed as `record[1]`, `record[2]`, *etc.*) if invoked simply as `LOAD CSV`
2. an object (items accessed as `record.col1`, `record.col2`, *etc.*) if invoked with `LOAD CSV WITH HEADERS`

```cypher
LOAD CSV FROM 'https://neo4j.com/docs/developer-manual/3.3/csv/artists.csv' AS line
CREATE (:ArtistNoHeaders { name: line[1], year: toInteger(line[2])})
                                                      
LOAD CSV WITH HEADERS FROM 'https://neo4j.com/docs/developer-manual/3.3/csv/artists-with-headers.csv' AS line
CREATE (:ArtistHeaders { name: line.Name, year: toInteger(line.Year)})
```

every single discussion I have ever seen about `LOAD CSV` has stressed at least 1 million times: when you load files of around 100k or more rows, preface your command with 

```cypher
USING PERIODIC COMMIT
LOAD CSV FROM ...
```

### functions

there are upwards of 80 [built-in functions](http://neo4j.com/docs/developer-manual/current/cypher/functions/) you can use to munge data within your queries, perform aggregate calculations, map or reduce calculations across lists, and much more.

in the interest of time we won't go into any in particular right now, but know that these are mostly operating in a *local* scope; that is, they will take a single subgraph matching a pattern and operate on it.

the exception to that (as with `sql`) is *aggregation* functions. there is no explicit aggregation clause (like `GROUP BY`) in `cypher`. instead, aggregation is implied based on the properties in the neighboring `RETURN` or `WITH` clause

### procedures

there is one clause we didn't discuss: `CALL`. we've actually already used this procedure, too, wayyyyyy back when we first opened the sandbox:

```cypher
CALL db.schema()
```

the `CALL` clause will invoke a *procedure*, which is a piece of built-in or user defined `java` code that extends the capabilities of the base `cypher` language to do something different: connect to some other system, or perform a complicated calculation.

several *procedures* are built-in and we can investigate what they are using a *procedure* (if you don't know what a dictionary is, me telling you to "look it up in the dictionary" won't be of much help will it?)

```cypher
CALL dbms.procedures()
```

the [`apoc`](https://neo4j-contrib.github.io/neo4j-apoc-procedures/) procedures (`apoc` stands for "a package of components" or "awesome procedures on cypher") are of particular interest, as that is where `neo4j` and the community have started to collect their advanced procedures, including optimized graph algorithm implementations

with `apoc`, calculating betweenness centrality and page rank directly from a cypher query is no problem:

```cypher
MATCH (node:Person)
WITH collect(node) AS nodes
CALL apoc.algo.betweenness(['ACTED_IN', 'DIRECTED', 'PRODUCED', 'WROTE'], nodes, 'BOTH') YIELD node, score
RETURN node, score
ORDER BY score DESC
```

```cypher
MATCH (node:Person)
WITH collect(node) AS nodes
CALL apoc.algo.pageRank(nodes) YIELD node, score
RETURN node, score
ORDER BY score DESC
```

additionally, `apoc` has implemented a procedure for reading and efficiently navigating `json` objects: 

```cypher
WITH "https://api.stackexchange.com/2.2/questions?pagesize=100&order=desc&sort=creation&tagged=neo4j&site=stackoverflow&filter=!5-i6Zw8Y)4W7vpy91PMYsKM-k9yzEsSC1_Uxlf" AS url
CALL apoc.load.json(url) YIELD value
UNWIND value.items AS item
RETURN item.title, item.owner, item.creation_date, keys(item)
```

I wonder if we'll find a way to use that...

there is *a lot* more than can be done using `cypher` -- go ahead and bookmark the [`cypher` reference cheatsheet](http://neo4j.com/docs/cypher-refcard/current/) now

## `cypher` in `python`

you're never gonna believe this.

we just spent all that time doing all that cool stuff via a web interface and now... we're just going to do it in `python` instead, because doign things from the terminal makes us feel warm and fuzzy inside

### library options

basically, there are two choices, and then others.

a developer named Nigel Small wrote an excellent `python` adapter for `neo4j` called [`py2neo`](http://py2neo.org/), so naturally `neo4j` hired him to have him write their other library, [`neo4j-driver`](https://neo4j.com/docs/api/python-driver/1.5/).

the community had collected pretty hard around `py2neo`, but I think the future is in `neo4j-driver` (if for no other reason that that the library is being developed in tandem with drivers for several other languages, so if you're switching languages or they ever make an `R` implementation the learning curve will be much less steep).

I'm going to recommend using `neo4j-driver`.

### `neo4j-driver`

first things first: [read the manual](https://neo4j.com/docs/api/python-driver/1.5/)

good, now that's out of the way. let's install the driver package. from a command line (your laptop or the `ec2` server, whichever you'd like to use

```bash
pip install neo4j-driver
```

if you experience permission errors, make sure you are using your *`conda` environment's* pip (try `which pip` to find out). you shoudl be using something like `~/miniconda3/bin/pip` or `~/miniconda3/envs/myenv/bin/pip`

if you started a `neo4j` sandbox with me earlier, there is a chance that some code using this library has already been generated for you.

+ head to https://neo4j.com/sandbox-v2/#
+ in the box for your sandbox, click the "Code" tab
+ select the `py` tab on that panel
+ copy and paste!
+ if there is nothing, launch the "Recommender" sandbox and repeat (the blank sandbox code snippers were having (no joke) `s3` permission issues at the time of writing this)

the code for your sandbox should look something like this:

```python
# pip install neo4j-driver

from neo4j.v1 import GraphDatabase, basic_auth

driver = GraphDatabase.driver(
    "bolt://YOUR_SERVER_IP:YOUR_PORT", 
    auth=basic_auth("neo4j", "YOUR_PASSWORD")
)
session = driver.session()

cypher_query = """
MATCH (n)
RETURN id(n) AS id
LIMIT $limit
"""

results = session.run(
    cypher_query,
    parameters={"limit": 10}
)

for record in results:
    print(record['id'])
```

let's walk through this code one block at a time.

after importing the necessary libraries, we create a *driver* object. the *driver* manages our *sessions* (individual statefull communications with the database) and creates *connections* (open channels through which *sessions* can talk to the database)

In [ ]:
from neo4j.v1 import GraphDatabase, basic_auth

driver = GraphDatabase.driver(
    "bolt://54.237.218.187:33448", 
    auth=basic_auth("neo4j", "rheostat-stick-cover")
)

after this, we create a *session*. a *session* is a stateful object which can make queries to the database (as needed) to execute queries and make transactions.

In [ ]:
session = driver.session()

next, we define our query. I've made a slight tweak to the query to include the node object itself (not just its `id`)

In [ ]:
cypher_query = """
MATCH (n)
RETURN id(n) AS id, n
LIMIT $limit
"""

note that we have yet another way of parameterizing our query strings here (for validation and sanitization): the `$` character.

the next stage is to simply execute the `cypher` code:

In [ ]:
results = session.run(
    cypher_query,
    parameters={"limit": 10}
)

the object returned by this `session.run` method is not a native `python` collection but a special object:

In [ ]:
results

the `results` object itself is natively iterable:

In [ ]:
for record in results:
    print(record['id'])

however, the `results.data()` method will often be more interesting to us. we need to re-query (the `results` item has been iterated so there would be no `data` to return)

In [ ]:
results = session.run(
    cypher_query,
    parameters={"limit": 10}
)
results.data()

In [ ]:
import pandas as pd

q = """
MATCH (p1:Person)-->(m1:Movie)<--(p2:Person)-->(m2:Movie)<--(p3:Person)
RETURN p1.name, m1.title, p2.name, m2.title, p3.name
"""
results = session.run(q)
pd.DataFrame(results.data()).head(20)

now compare those eight lines to the code we wrote before to do the same thing for *only* actors in the relational setting:

```python
qry = """
SELECT
    p1.name AS p1_name
    , m1.title AS m1_title
    , p2.name AS p2_name
    , m2.title AS m2_title
    , p3.name AS p3_name
FROM
    person AS p1
    JOIN acted_in AS a1
    ON
        p1.id = a1.person
    JOIN movie AS m1
    ON
        a1.movie = m1.id
    JOIN acted_in AS a2
    ON
        m1.id = a2.movie
        AND a1.person != a2.person
    JOIN person AS p2
    ON 
        a2.person = p2.id
    JOIN acted_in AS a3
    ON
        p2.id = a3.person
    JOIN movie AS m2
    ON
        a3.movie = m2.id
    JOIN acted_in AS a4
    ON
        m2.id = a4.movie
        AND a3.person != a4.person
        AND a1.person != a4.person
    JOIN person AS p3
    ON 
        a4.person = p3.id
"""

with psycopg2.connect(host=host, port=port, user=user, dbname=dbname, password=password) as conn:
    partners = pd.read_sql(qry, conn)
partners.head(10)
```

open links:

+ neo sandbox: https://neo4j.com/sandbox-v2/#
+ movie datasets
    + http://www.omdbapi.com/
    + https://grouplens.org/datasets/movielens/
+ csv bulk load example: https://neo4j.com/developer/guide-import-csv/
    + could load northwind example dataset: https://neo4j.com/developer/guide-importing-data-and-etl/
    + also northwind: https://code.google.com/archive/p/northwindextended/downloads
    + another alternative: consumer complaints: https://catalog.data.gov/dataset/consumer-complaint-database
+ paradise papers: https://www.icij.org/investigations/paradise-papers/explore-politicians-paradise-papers/

<!--div align="center">***DROP joke WHERE is_bad***</div>
<img align="middle" src=""></img-->

# END OF LECTURE

next lecture: [`hadoop`](011_hadoop.ipynb)